# DETECCIÓN DE MOVIMIENTO CON OPENCV

## FRAME DIFFERENCING

In [15]:
import cv2
import numpy as np

import cv2

# Captura de video desde un archivo
ruta_del_video = "FLUJO OPTICO/MAN_WALK.mp4"
cap = cv2.VideoCapture(ruta_del_video)
#cap = cv2.VideoCapture(0)
# Leer el primer fotograma
ret, frame_anterior = cap.read()
frame_anterior = cv2.cvtColor(frame_anterior, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    ret, frame_actual = cap.read()
    if not ret:
        break
    
    # Convertir a escala de grises
    frame_actual = cv2.cvtColor(frame_actual, cv2.COLOR_BGR2GRAY)
    
    # Calcular la diferencia absoluta entre fotogramas
    diff = cv2.absdiff(frame_anterior, frame_actual)
    
    # Aplicar umbralización para resaltar el movimiento
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    # Mostrar resultados
    cv2.imshow('Diferencia de fotogramas', diff)
    cv2.imshow('Movimiento detectado', thresh)
    
    # Actualizar el fotograma anterior
    frame_anterior = frame_actual.copy()
    
    # Salir con la tecla 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Este método es simple:

Se captura el video.
Se obtiene el fotograma actual y el anterior.
Se calcula la diferencia entre ambos.
Se aplica umbralización para resaltar el movimiento.

## SUTRACCION DE FONDO

In [ ]:
import cv2
import numpy as np

# Captura de video desde un archivo
ruta_del_video = "FLUJO OPTICO/MAN_WALK.mp4"
cap = cv2.VideoCapture(ruta_del_video)

# Crear el sustractor de fondo (GMM)
fgbg = cv2.createBackgroundSubtractorMOG2()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Aplicar la sustracción de fondo
    fgmask = fgbg.apply(frame)
    
    # Mostrar resultados
    cv2.imshow('Frame Original', frame)
    cv2.imshow('Sustracción de Fondo', fgmask)
    
    # Salir con la tecla 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## FLUJO OPTICO

In [13]:
import cv2
import numpy as np


# Captura de video desde un archivo
#ruta_del_video = "FLUJO OPTICO/MAN_WALK.mp4"
#cap = cv2.VideoCapture(ruta_del_video)
cap = cv2.VideoCapture(0)
# Parámetros para el flujo óptico de Farnebäck
params = dict(pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

# Leer el primer fotograma
ret, frame_anterior = cap.read()
gray_anterior = cv2.cvtColor(frame_anterior, cv2.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame_anterior)
hsv[..., 1] = 255  # Saturación máxima para visualizar mejor

while cap.isOpened():
    ret, frame_actual = cap.read()
    if not ret:
        break
    
    gray_actual = cv2.cvtColor(frame_actual, cv2.COLOR_BGR2GRAY)
    
    # Calcular flujo óptico de Farnebäck
    flow = cv2.calcOpticalFlowFarneback(gray_anterior, gray_actual, None, **params)
    
    # Convertir a ángulo y magnitud para visualización
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2  # Convertir ángulo a escala de colores
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)  # Normalizar magnitud
    
    # Convertir HSV a BGR para mostrar
    bgr_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # Mostrar resultados
    cv2.imshow('Frame Original', frame_actual)
    cv2.imshow('Flujo Óptico', bgr_flow)
    
    # Actualizar el fotograma anterior
    gray_anterior = gray_actual.copy()
    
    # Salir con la tecla 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## DETECCION DE ESQUINAS

In [12]:
import cv2
import numpy as np

# Captura de video desde un archivo
#ruta_del_video = "FLUJO OPTICO/MAN_WALK.mp4"
#cap = cv2.VideoCapture(ruta_del_video)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detección de esquinas con Harris
    gray = np.float32(gray)
    corners = cv2.cornerHarris(gray, 2, 3, 0.04)
    frame[corners > 0.01 * corners.max()] = [0, 0, 255]  # Marcar esquinas en rojo
    
    # Mostrar resultados
    cv2.imshow('Detección de Esquinas', frame)
    
    # Salir con la tecla 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [11]:
# Canny + Contornos
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)  # Dibujar contornos en verde
    
    # Mostrar resultados
    cv2.imshow('Canny + Contornos', frame)
    
    # Salir con la tecla 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()